# Pre-process non-labeled dataset

## Convert to nifti

In [ ]:
import os, glob
import subprocess
import numpy as np

input_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset/'
output_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop
# i=0
for folder1 in sorted(os.listdir(input_dir)):
    # i+=1
    filename = folder1
    for folder2 in os.listdir(input_dir+folder1):
        # if i==10: break
        if 't1' in folder2 and not folder2.startswith('.'):
            input_dicom_folder = input_dir+folder1+'/'+folder2
            
            # output
            output_nifti_folder = output_dir+folder1
            if not os.path.exists(output_nifti_folder):
                os.makedirs(output_nifti_folder)

            # dcm2niix
            # dicom2nifti.dicom_series_to_nifti(input_dicom_folder, output_nifti_folder+filename, reorient_nifti=True)
            cmd = ["dcm2niix", "-f", filename, "-z", "y", "-o", output_nifti_folder, input_dicom_folder]
            process = subprocess.Popen(cmd, stdout=subprocess.PIPE)  # pass the list as input to Popen
            _ = process.communicate()[0]  # the [0] is to return just the output, because otherwise it would be outs, errs = proc.communicate()
            
            # Dealing with files in that folder
            # for f in glob.glob(input_dir+folder1+'/anat/'+folder1+'_T1.nii.gz'):
            #     os.remove(f)

## Crop eye center

In [ ]:
import numpy
import os,sys
import nibabel as nib
import SimpleITK as sitk
import pandas as pd
import numpy as np

def cropVolumeImage(image,x,y,z,border1,border2,border3):
    
    xSize, ySize, zSize = image.GetSize()
    #xMin,yMin,zMin = image.TransformPhysicalPointToIndex([x-border,y-border,z-border])
    #xMax,yMax,zMax = image.TransformPhysicalPointToIndex([x+border,y+border,z+border])

    #x,y,z = image.TransformPhysicalPointToIndex([x,y,z])
    xMin = x - int(border1/2) 
    xMax = x + int(border1/2) 

    yMin = y - int(border2/2) 
    yMax = y + int(border2/2)  

    zMin = z - int(border3/2) 
    zMax = z + int(border3/2)
    
    # Define limits
    if xMin< 0 :
        xMin = 0  
    if xMax> xSize:
        xMax = xSize

    if yMin< 0 :
        yMin = 0
    if yMax > ySize:
        yMax = ySize

    if zMin< 0 :
        zMin = 0
    if zMax > zSize:
        zMax = zSize

    imageReturn = image[int(xMin):int(xMax),int(yMin):int(yMax),int(zMin):int(zMax)]
    return imageReturn  

# Paths
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti/'
output_path = main_path + 'non_labeled_dataset_nifti_cropped/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Variables to crop 
eye_center = np.array([112, 173, 67]) # Mean coordinates of the center of the right eye (base of the cornea to catch optic nerve too)
croping_cube = np.array([96, 96, 96])

i = 0
for folder1 in sorted(os.listdir(input_path)):
    filename = folder1
    # Path to the folder with images
    input_image_path = input_path + folder1 + '/' + folder1 + '.nii.gz'
    output_image_path = output_path + folder1 + '.nii.gz'
    print(input_image_path)

    input_image = sitk.ReadImage(input_image_path)

    cropped_image = cropVolumeImage(input_image, eye_center[0], eye_center[1], eye_center[2], croping_cube[0], croping_cube[1], croping_cube[2])

    sitk.WriteImage(cropped_image, output_image_path)

    i+=1
    if (i==10):
        break

## Registration 1 - Full images

In [ ]:
import os, glob

# Paths - full image (1st iteration)
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti/'
output_path = main_path + 'non_labeled_dataset_nifti_reg/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0.nii.gz' # full image template - fixed image (reference)
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/sub-29_labels2template5_2.nii.gz'

# i = 0
for folder1 in sorted(os.listdir(input_path)):
    if folder1 not in sorted(os.listdir(output_path)):
    
        # Full images paths
        input_image_path = input_path + folder1 + '/' + folder1 + '.nii.gz' # moving image - full
        output_image_path = output_path + folder1 + '/'

        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + input_image_path                     + \
        ' -f ' + template                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        # os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  template_labels                                    + \
        ' -r ' +  input_image_path                                   + \
        ' -t ' + '[' + output_image_path + '0GenericAffine.mat, 1 ]' + \
        ' -t ' + output_image_path + '1InverseWarp.nii.gz'           + \
        ' -n ' + 'MultiLabel'                                        + \
        ' -o ' +  output_image_path + 'labels.nii.gz'                + \
        ' --float 0 --verbose 1'
        # print(command2)
        # os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

        # i+=1
        # if (i==1):
        #     break

## Crop

In [ ]:
import numpy as np
import os,sys
import nibabel as nib
import SimpleITK as sitk
import pandas as pd

# Paths
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_image_path = main_path + 'non_labeled_dataset_nifti/'
input_label_path = main_path + 'non_labeled_dataset_nifti_reg/'
output_path = main_path + 'non_labeled_dataset_nifti_cropped/'

# i=0
for folder1 in sorted(os.listdir(input_label_path)):
    if folder1+'_cropped.nii.gz' not in list(sorted(os.listdir(output_path))):

        image_path = input_image_path + folder1 + '/' + folder1 + '.nii.gz' # image
        labels_path = input_label_path + folder1 + '/labels.nii.gz' # labels
        bound = 15 # boundary for the bounding box (margins)

        image = sitk.ReadImage(image_path)
        all_segments = sitk.ReadImage(labels_path)
        image_x_size, image_y_size, image_z_size = image.GetSize()
        print(f"image_x_size {image_x_size} image_y_size {image_y_size} image_z_size {image_z_size}")

        # Mask
        all_segments_mask = all_segments > 0
        # sitk.WriteImage(all_segments_mask, base_dir+folder+'/input/'+folder+'_labels_mask.nii.gz')

        # Bounding box
        lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
        lsif.Execute(image, all_segments_mask) # Mask! Where all the labels are 1!
        bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
        print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
        bounding_box_expanded = bounding_box.copy()
        bounding_box_expanded[0::2] -= bound # even indexes
        bounding_box_expanded[1::2] += bound # odd indexes
        print(f"Expanded bounding box: {bounding_box_expanded}")

        # Limits
        if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
        if bounding_box_expanded[1] > image_x_size: bounding_box_expanded[1] = image_x_size
        if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
        if bounding_box_expanded[3] > image_y_size: bounding_box_expanded[3] = image_y_size
        if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
        if bounding_box_expanded[5] > image_z_size: bounding_box_expanded[5] = image_z_size
        print(f"Expanded bounding box after limits: {bounding_box_expanded}")

        # Crop
        image_crop = image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                            int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                            int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        sitk.WriteImage(image_crop, output_path + folder1 + '_cropped.nii.gz')

        # i+=1
        # if (i==1):
        #     break             

## Registration 2 - Cropped images

In [ ]:
# Paths - cropped images (2nd iteration)
main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
input_path = main_path + 'non_labeled_dataset_nifti_cropped/'
output_path = main_path + 'non_labeled_dataset_nifti_reg_2/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0_cropped_15vox.nii.gz' # cropped image template
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/Probability_Maps/prob_map_cropped_th0_2.nii.gz'

# i = 0
for folder1 in sorted(os.listdir(input_path)):
    # if folder1.split('_')[0] not in sorted(os.listdir(output_path)):
    if folder1 == '2022160100031_cropped.nii.gz':
        # Cropped images paths
        filename = folder1.split('_')[0]
        input_image_path = input_path + folder1 # moving image - cropped
        output_image_path = output_path + filename + '/'

        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + input_image_path                     + \
        ' -f ' + template                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  template_labels                                    + \
        ' -r ' +  input_image_path                                   + \
        ' -t ' + '[' + output_image_path + '0GenericAffine.mat, 1 ]' + \
        ' -t ' + output_image_path + '1InverseWarp.nii.gz'           + \
        ' -n ' + 'MultiLabel'                                        + \
        ' -o ' +  output_image_path + 'labels.nii.gz'                + \
        ' --float 0 --verbose 1'
        # print(command2)
        os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

        # i+=1
        # if (i==99):
        #     break

# nnUnet results

Registration of atlas and nnUNet labels to crop and allign them to the same space

In [ ]:
import os

# Paths
reg_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2/'
nnunet_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing/'
output_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing_cropped_reg/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

i = 0
for folder1 in sorted(os.listdir(reg_dir)):
    # if folder1 not in sorted(os.listdir(output_path)):
    if folder1 == '2022160101206':
        # Images' paths
        fixed_image_path = f'{reg_dir}{folder1}/labels.nii.gz' # fixed image (atlas)
        moving_image_path = f'{nnunet_dir}AEye_{folder1}.nii.gz' # moving image (nnunet)
        output_image_path = f'{output_path}{folder1}/'
        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyNQuick.sh -d 3' + \
        ' -m ' + moving_image_path                     + \
        ' -f ' + fixed_image_path                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  moving_image_path                                    + \
        ' -r ' +  fixed_image_path                                   + \
        ' -t ' + output_image_path + '1Warp.nii.gz'           + \
        ' -t ' + output_image_path + '0GenericAffine.mat' + \
        ' -o ' +  output_image_path + 'cropped_alligned.nii.gz'                + \
        ' --float 0 --verbose 0' +\
        ' -n ' + 'Multilabel'                                        
        # print(command2)
        os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)

        i+=1
        # if (i==1):
        #     break

# New data

Convert to nifti

In [ ]:
import os
import subprocess

input_dir = '/home/jaimebarranco/Downloads/MREye_Studybrain_1/'
output_dir = '/home/jaimebarranco/Downloads/output_atlas/nifti/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop
for folder in sorted(os.listdir(input_dir)):
    filename = folder
    input_dicom_folder = input_dir + folder
    
    # output
    output_nifti_folder = output_dir+folder
    if not os.path.exists(output_nifti_folder):
        os.makedirs(output_nifti_folder)

    # dcm2niix
    # dicom2nifti.dicom_series_to_nifti(input_dicom_folder, output_nifti_folder+filename, reorient_nifti=True)
    cmd = ["dcm2niix", "-f", filename, "-z", "y", "-o", output_nifti_folder, input_dicom_folder]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE)  # pass the list as input to Popen
    _ = process.communicate()[0]  # the [0] is to return just the output, because otherwise it would be outs, errs = proc.communicate()
    # Dealing with files in that folder
    # for f in glob.glob(input_dir+folder1+'/anat/'+folder1+'_T1.nii.gz'):
    #     os.remove(f)

Registration 1 - Full images

In [ ]:
os.environ['PATH'] += ':/opt/ANTs/bin' # ensure the ANTs binaries are in $PATH
os.environ['ANTSPATH'] = '/opt/ANTs/bin/'
# os.system("echo $PATH")
# os.system("which antsApplyTransforms")
# os.system("antsApplyTransforms -h")
print(os.environ['PATH'])

In [ ]:
import os, subprocess

# Paths - full image (1st iteration)
input_path = '/home/jaimebarranco/Downloads/output_atlas/nifti/'
output_path = '/home/jaimebarranco/Downloads/output_atlas/reg1/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0.nii.gz' # full image template - fixed image (reference)
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/Probability_Maps/prob_map_th0.nii.gz' # mix of labels from the 5 subjects

for folder in sorted(os.listdir(input_path)):
    
    # Full images paths
    input_image_path = input_path + folder + '/' + folder + '.nii.gz' # moving image - full
    output_image_path = output_path + folder + '/'

    if not os.path.exists(output_image_path):
        os.makedirs(output_image_path)

    # antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
    command1 = ['antsRegistrationSyNQuick.sh', 
                '-d', '3',
                '-m', input_image_path,
                '-f', template,
                '-t', 's',
                '-o', output_image_path,
                '-n', '16']
    print(command1)
    # os.system(command1)
    process = subprocess.Popen(command1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # antsApplyTransforms with inverse transform to get the template labels into subject space
    command2 = ['antsApplyTransforms', '-d', '3',
                '-i', template_labels,
                '-r', input_image_path,
                '-t', '[' + output_image_path + '0GenericAffine.mat, 1 ]',
                '-t', output_image_path + '1InverseWarp.nii.gz',
                '-n', 'MultiLabel',
                '-o', output_image_path + 'labels.nii.gz',
                '--float', '0',
                '--verbose', '1']
    # os.system(command2)
    process = subprocess.Popen(command2, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(f'Error: {stderr.decode()}')
    else:
        print(stdout.decode())

    # Dealing with files in that folder
    # for f in glob.glob(output_image_path + 'labels.nii.gz'):
    #     os.remove(f)

Allignment image - labels

In [ ]:
import os
import nibabel as nb

# Paths
input_path = '/home/jaimebarranco/Downloads/output_atlas/nifti/'
output_path = '/home/jaimebarranco/Downloads/output_atlas/reg1/alligned/'
labels_path = '/home/jaimebarranco/Downloads/output_atlas/reg1/'

for folder in sorted(os.listdir(input_path)):

    # Load reference image
    image = nb.load(input_path + folder + '/' + folder + '.nii.gz') # load image

    # Load t1 image (not alligned)
    t1 = nb.load(labels_path + folder + '/' + 'labels.nii.gz')
    print(t1.affine)

    # Copy affine from template to t1
    nii = nb.Nifti1Image(t1.dataobj, image.affine, image.header)
    print(t1.affine)

    # Output
    output_image_path = output_path + folder
    if not os.path.exists(output_image_path):
        os.makedirs(output_image_path)
    nii.to_filename(output_image_path + '/' + 'labels_aff.nii.gz')
    
    # Dealing with files in that folder
    # for f in glob.glob(base_dir+folder+'/input/'+folder+'_T1_oriented_hdr.nii.gz'):
    #     os.remove(f)

Crop

In [ ]:
import numpy as np
import os
import SimpleITK as sitk

# Paths
input_image_path = '/home/jaimebarranco/Downloads/output_atlas/nifti/'
input_label_path = '/home/jaimebarranco/Downloads/output_atlas/reg1/alligned/'
output_path = '/home/jaimebarranco/Downloads/output_atlas/cropped/'

for folder in sorted(os.listdir(input_label_path)):

    image_path = input_image_path + folder + '/' + folder + '.nii.gz' # image
    labels_path = input_label_path + folder + '/' + 'labels_aff.nii.gz' # labels
    bound = 50 # boundary for the bounding box (margins)

    image = sitk.ReadImage(image_path)
    all_segments = sitk.ReadImage(labels_path)
    image_x_size, image_y_size, image_z_size = image.GetSize()
    print(f"image_x_size {image_x_size} image_y_size {image_y_size} image_z_size {image_z_size}")

    # Mask
    all_segments_mask = all_segments > 0
    # sitk.WriteImage(all_segments_mask, base_dir+folder+'/input/'+folder+'_labels_mask.nii.gz')

    # Bounding box
    lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
    lsif.Execute(image, all_segments_mask) # Mask! Where all the labels are 1!
    bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
    print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
    bounding_box_expanded = bounding_box.copy()
    bounding_box_expanded[0::2] -= bound # even indexes
    bounding_box_expanded[1::2] += bound # odd indexes
    print(f"Expanded bounding box: {bounding_box_expanded}")

    # Limits
    if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
    if bounding_box_expanded[1] > image_x_size: bounding_box_expanded[1] = image_x_size
    if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
    if bounding_box_expanded[3] > image_y_size: bounding_box_expanded[3] = image_y_size
    if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
    if bounding_box_expanded[5] > image_z_size: bounding_box_expanded[5] = image_z_size
    print(f"Expanded bounding box after limits: {bounding_box_expanded}")

    # Crop
    image_crop = image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                        int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                        int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z
    if not os.path.exists(output_path + folder):
        os.makedirs(output_path + folder)
    sitk.WriteImage(image_crop, output_path + folder + '/' + folder + '_cropped.nii.gz')            

Registration 2 - Cropped images

In [ ]:
import os

# Paths - cropped images (2nd iteration)
input_path = '/home/jaimebarranco/Downloads/MREye_Studybrain_1_segmentations/atlas/cropped/'
output_path = '/home/jaimebarranco/Downloads/MREye_Studybrain_1_segmentations/atlas/reg2_SyN/'
template = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/template0_cropped_15vox.nii.gz' # cropped image template
template_labels = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/Probability_Maps/prob_map_cropped_th0_2.nii.gz'

for folder in sorted(os.listdir(input_path)):
        # Cropped images paths
        input_image_path = input_path + folder + '/' + folder + '_cropped.nii.gz' # moving image - cropped
        output_image_path = output_path + folder + '/'

        if not os.path.exists(output_image_path):
            os.makedirs(output_image_path)

        ## antsRegistrationSyNQuick # s: rigid + affine + deformable syn (3 stages)
        command1 = 'antsRegistrationSyN.sh -d 3' + \
        ' -m ' + input_image_path                     + \
        ' -f ' + template                             + \
        ' -t ' + 's'                                  + \
        ' -o ' + output_image_path                    + \
        ' -n ' + '16'
        # print(command1)
        os.system(command1)

        # antsApplyTransforms with inverse transform to get the template labels into subject space
        command2 = 'antsApplyTransforms -d 3 '                       + \
        ' -i ' +  template_labels                                    + \
        ' -r ' +  input_image_path                                   + \
        ' -t ' + '[' + output_image_path + '0GenericAffine.mat, 1 ]' + \
        ' -t ' + output_image_path + '1InverseWarp.nii.gz'           + \
        ' -n ' + 'MultiLabel'                                        + \
        ' -o ' +  output_image_path + 'labels.nii.gz'                + \
        ' --float 0 --verbose 1'
        # print(command2)
        os.system(command2)

        # Dealing with files in that folder
        # for f in glob.glob(output_image_path + 'labels.nii.gz'):
        #     os.remove(f)